In [1]:
using PyPlot

INFO: Loading help data...


In [47]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

elapsed time: 1.072894188 seconds (471040 bytes allocated)


In [1]:
addprocs();
function sendto(p::Int; args...)
    for (nm, val) in args
        @spawnat(p, eval(Main, Expr(:(=), nm, val)))
    end
end


function sendto(ps::Vector{Int}; args...)
    for p in ps
        sendto(p; args...)
    end
end

getfrom(p::Int, nm::Symbol; mod=Main) = fetch(@spawnat(p, getfield(mod, nm)))



getfrom (generic function with 1 method)

In [2]:
using SortSpikes, ExtractSpikes, DistributedArrays

In [4]:

s=Any[SpikeDetection() for i=1:64]
c=Any[Cluster() for i=1:64]
electrode=Any[zeros(Int64,2) for i=1:64]
neuronnum=Any[zeros(Int64,2) for i=1:64]
be=ones(Int64,20000,64)
for i=1:64
    be[200:700:end,i]=10*be[200:700:end,i]
end
@time onlineCal(be,c,s)
onlineSort([1,20000],be,c,s,electrode,neuronnum)



 137.937 milliseconds (101 k allocations: 36699 KB, 6.42% gc time)


In [9]:
#create local copies for each processor
sendto(2, s=Any[SpikeDetection() for i=1:32])
sendto(3, s=Any[SpikeDetection() for i=1:32])
sendto(2, c=Any[Cluster() for i=1:32])
sendto(3, c=Any[Cluster() for i=1:32])

#this is read only, so should work fine, but might have to be shared array
be=ones(Int64,20000,64)
for i=1:64
    be[200:700:end,i]=10*be[200:700:end,i]
end

#each processor should run independently
function main()
for p in workers()
    @spawnat p begin
        onlineCal_par(be,c,s,p-1,2)
    end
end
end




main (generic function with 1 method)

In [10]:
@time main()

  85.300 milliseconds (2209 allocations: 123 KB)
